# 데이터 작업
- dataset : 샘플, 정답
- dataloader : dataset을 순회 가능한 객체로 감싼 것

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

Using downloaded and verified file: data/FashionMNIST/raw/train-images-idx3-ubyte.gz
Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw
Using downloaded and verified file: data/FashionMNIST/raw/train-labels-idx1-ubyte.gz
Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Using downloaded and verified file: data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz
Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw
Using downloaded and verified file: data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


/Users/sherry/opt/anaconda3/envs/torch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1607370126481/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### dataloader 지원
- 자동화된 배치
- 샘플링
- shuffle
- multiprocess data loading

In [15]:
batch_size = 64

# 데이터 로더 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X[N, C, H, W] : {X.shape}') # batch_size, channel, height, width
    print(f'Shape of y: {y.shape} / {y.dtype}')
    break

Shape of X[N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) / torch.int64


### 모델 만들기
- nn.Module을 상속받는 class로 생성
- init에서 신경망의 계층을 정의, forward에서 신경에 데이터를 어떻게 전달할지 지정


In [5]:
#gpu 또는 cpu 정의
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using : {device}device')

Using : cpudevice


In [23]:
# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 모델 매개변수 최적화하기
- 모델 학습을 위해서 손실함수(loss function)과 옵티마이저(OPTIMIZER) 필요
- 각 학습 단계에서 모델은 학습 데이터에(배치) 대한 예측 수행 -> 예측 오류 역전파 -> 모델의 매개변수 조정

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)



In [26]:
def train(dataloader, model, loss_fn, optimzer):
    size = len(dataloader.dataset) # 전체 데이터 개수
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimzer.zero_grad()
        loss.backward()
        optimzer.step()

        if batch % 100==0:
            loss, current = loss.item(), batch*len(X)
            print(f'loss: {loss:7f} [{current:5d}/{size :5d}]')

# 테스트 데이터 셋으로 모델의 성능 확인
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# 각 에폭(Epoch)에서 모델은 더 나은 예측을 위해 매개 변수를 학습.

epoch = 5
for t in range(epoch):
    print(f'Epoch : {t+1} ------------------- ')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print('Done!')



Epoch : 1 ------------------- 
loss: 2.302335 [    0/60000]
loss: 2.305166 [ 6400/60000]
loss: 2.304675 [12800/60000]
loss: 2.307642 [19200/60000]
loss: 2.303192 [25600/60000]
loss: 2.300145 [32000/60000]
loss: 2.298880 [38400/60000]
loss: 2.291548 [44800/60000]
loss: 2.302987 [51200/60000]
loss: 2.299479 [57600/60000]
Test Error: 
 Accuracy: 17.1%, Avg loss: 2.303559 

Epoch : 2 ------------------- 
loss: 2.302335 [    0/60000]
loss: 2.305166 [ 6400/60000]
loss: 2.304675 [12800/60000]
loss: 2.307642 [19200/60000]
loss: 2.303192 [25600/60000]
loss: 2.300145 [32000/60000]
loss: 2.298880 [38400/60000]
loss: 2.291548 [44800/60000]
loss: 2.302987 [51200/60000]
loss: 2.299479 [57600/60000]
Test Error: 
 Accuracy: 17.1%, Avg loss: 2.303559 

Epoch : 3 ------------------- 
loss: 2.302335 [    0/60000]
loss: 2.305166 [ 6400/60000]
loss: 2.304675 [12800/60000]
loss: 2.307642 [19200/60000]
loss: 2.303192 [25600/60000]
loss: 2.300145 [32000/60000]
loss: 2.298880 [38400/60000]
loss: 2.291548 [4480

### 모델 저장하기

- 내부 상태 사전(internal state dictionary, 모델의 매개변수 포함) 직렬화

In [28]:
torch.save(model.state_dict(), './model/model.pth')
print('saved pytorch model')

saved pytorch model


### 모델 불러오기
- 모델을 불러오는과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함됨


In [29]:
model = NeuralNetwork()
model.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [30]:
# 모델을 사용한 예측
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
X,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(X)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Prediction : {predicted} / Actual :{actual}')



Prediction : Ankle boot / Actual :Ankle boot


In [ ]:
# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),

        )